# Factors

In [1]:
import pandas as pd
from scipy.optimize import minimize
import statsmodels.api as sm

#%pylab inline

c:\users\samth\miniconda3\envs\econ\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# pd.read_csv("C:/Data/Thesis/Factors.csv").set_index("date")

F = pd.read_stata("C:/Data/Thesis/Factors.dta").set_index("date")

In [3]:
ff2015 = ["rm", "smb", "hmlo", "rmw", "cma"]
ff2016a = ["rm", "smb", "hml", "RMWr", "cma"]
ff2016b = ["rm", "smb", "hml", "RMWc", "cma"]
bs2015 = ["rm", "smb", "HMLm", "wml", "RMWc"]

## Summary Statistics

### French Dartmouth Website

In [ ]:
x = ["SMB", "smb", "HML", "hml", "RMW", "rmw", "CMA", "cma", "WML", "wml"]
tbl = pd.concat({"Mean":  F[x].mean(),
                 "Stdev": F[x].std(),
                 "N":     F[x].count()}, axis="columns")

tbl["t-stat"] = tbl["Mean"] / (tbl["Stdev"] / pd.np.sqrt(tbl["N"]))
round(tbl.drop("N", axis=1).transpose(), 2)

In [ ]:
x = ["HMLs", "HMLb", "RMWs", "RMWb", "CMAs", "CMAb", "WMLs", "WMLb"]
tbl = pd.concat({"Mean":  F[x].mean(),
                 "Stdev": F[x].std(),
                 "N":     F[x].count()}, axis="columns")

tbl["t-stat"] = tbl["Mean"] / (tbl["Stdev"] / pd.np.sqrt(tbl["N"]))
round(tbl.drop("N", axis=1).transpose(), 2)

### Fama French (2016) _Choosing Factors_

In [ ]:
x = ["HML", "HMLs", "HMLb", "RMWr", "RMWrs", "RMWrb",
     "RMWc", "RMWcs", "RMWcb", "CMA", "CMAs", "CMAb"]
tbl = pd.concat({"Mean":  F[:"2015"][x].mean(),
                 "Stdev": F[:"2015"][x].std(),
                 "N":     F[:"2015"][x].count()}, axis="columns")

tbl["t-stat"] = tbl["Mean"] / (tbl["Stdev"] / pd.np.sqrt(tbl["N"]))
round(tbl.drop("N", axis=1).transpose(), 2)

### Table 1

In [ ]:
x = ["rm", "smb", "hml", "HMLm", "RMWc", "cma", "wml"]
tbl = pd.concat({"Mean":  F[x].mean(),
                 "Stdev": F[x].std(),
                 "N":     F[x].count()}, axis="columns")

tbl["t-stat"] = tbl["Mean"] / (tbl["Stdev"] / pd.np.sqrt(tbl["N"]))
tbl["Sh"] = tbl["Mean"] / tbl["Stdev"]
tbl["Sh2"] = tbl["Sh"]**2
round(tbl.drop("N", axis=1).transpose(), 2)

### Value

In [ ]:
x = ["HMLs", "HMLb", "HML", "HMLcs", "HMLcb", "HMLc",
     "HMLms", "HMLmb", "HMLm"]
tbl = pd.concat({"Mean":  F[x].mean(),
                 "Stdev": F[x].std(),
                 "N":     F[x].count()}, axis="columns")

tbl["t-stat"] = tbl["Mean"] / (tbl["Stdev"] / pd.np.sqrt(tbl["N"]))
round(tbl.drop("N", axis=1).transpose(), 2)

### Profit

In [ ]:
x = ["RMWs", "RMWb", "RMW", "RMWrs", "RMWrb", "RMWr",
     "RMWgs", "RMWgb", "RMWg", "RMWcs", "RMWcb", "RMWc"]
tbl = pd.concat({"Mean":  F[x].mean(),
                 "Stdev": F[x].std(),
                 "N":     F[x].count()}, axis="columns")

tbl["t-stat"] = tbl["Mean"] / (tbl["Stdev"] / pd.np.sqrt(tbl["N"]))
round(tbl.drop("N", axis=1).transpose(), 2)

## Sharpe Ratio

We are inverting a matrix - check the condition number.

In [ ]:
r = F[:"2015"][ff2016a].mean()
Vf = F[:"2015"][ff2016a].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

In [ ]:
r = F[:"2015"][ff2016a].mean().as_matrix()
Vf = F[:"2015"][ff2016a].cov().as_matrix()

r @ pd.np.linalg.pinv(Vf) @ r

In [ ]:
r = F[:"2015"][ff2016b].mean()
Vf = F[:"2015"][ff2016b].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

In [ ]:
r = F[:"2015"][bs2015].mean()
Vf = F[:"2015"][bs2015].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

In [ ]:
r = F[:"2016"][bs2015].mean()
Vf = F[:"2016"][bs2015].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

In [ ]:
r = F["2000":"2016"][ff2016b].mean()
Vf = F["2000":"2016"][ff2016b].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

In [ ]:
r = F["2000":"2016"][bs2015].mean()
Vf = F["2000":"2016"][bs2015].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

In [ ]:
r = F["1963":"1999"][ff2016b].mean()
Vf = F["1963":"1999"][ff2016b].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

In [ ]:
r = F["1963":"1999"][bs2015].mean()
Vf = F["1963":"1999"][bs2015].cov()

r.dot(pd.np.linalg.pinv(Vf)).dot(r)

## Weights

### Sharpe Ratio

$$E(R_p) = w'\bar{R}$$
$$Var(R_p) = w'V_Rw$$
$$Sh^2(f) = \bar{f}'V_f^{-1}\bar{f}$$

In [ ]:
def ERp(w, R):
    ER = pd.np.mean(R, axis=0)
    return w.transpose() @ ER

In [ ]:
def VRp(w, R):
    CoVR = pd.np.cov(R, rowvar=False)
    return w.transpose() @ CoVR @ w

In [ ]:
R = F[:"2016"][bs2015].as_matrix()
CoVR = pd.np.cov(R, rowvar=False)
F[:"2016"][bs2015].cov().as_matrix() == CoVR

In [ ]:
def Sh2(f):
    Ef = pd.np.mean(f, axis=0)
    CoVf = pd.np.cov(f, rowvar=False)
    return Ef.transpose() @ pd.np.linalg.pinv(CoVf) @ Ef

In [ ]:
def SR(w, R):
    return (ERp(w, R) / pd.np.sqrt(VRp(w, R)))

In [ ]:
R = F[bs2015].as_matrix()
w = pd.np.matrix([.2, .2, .2, .2, .2]).transpose()

In [ ]:
w.transpose() @ pd.np.mean(R, axis=0), pd.np.mean(w.transpose() @ R.transpose())

In [ ]:
w.shape, R.shape

In [ ]:
ERp(w, R), VRp(w, R), SR(w, R), pd.np.sqrt(Sh2(R))

### Objective Function

In [ ]:
def obj_SR(*args, **kwargs):
    return -SR(*args, **kwargs)

In [ ]:
obj_SR(w, R)

In [ ]:
# equality constraint functions are set to 0
cons = ({"type": "eq", "fun": lambda w: pd.np.sum(w) - 1},)

In [ ]:
R = F[:"2016"][bs2015].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
BS2015 = pd.Series(res["x"] @ R.transpose(), index=F.index)
bs2015, res["x"].round(2)

In [ ]:
R = F[:"2016"][ff2015].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
FF2015 = pd.Series(res["x"] @ R.transpose(), index=F.index)
ff2015, res["x"].round(2)

In [ ]:
R = F[:"2016"][ff2016a].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
FF2016a = pd.Series(res["x"] @ R.transpose(), index=F.index)
ff2016a, res["x"].round(2)

In [ ]:
R = F[:"2016"][ff2016b].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
FF2016b = pd.Series(res["x"] @ R.transpose(), index=F.index)
ff2016b, res["x"].round(2)

In [ ]:
models = pd.DataFrame(
    {"BS2015": BS2015,
     "FF2015": FF2015,
     "FF2016a": FF2016a,
     "FF2016b": FF2016b,
     "Rm": F["rm"]}
)

## Table 1

In [ ]:
tbl = pd.concat({"Mean":  models.mean(),
                 "Stdev": models.std(),
                 "N":     models.count()}, axis="columns")

tbl["t-stat"] = tbl["Mean"] / (tbl["Stdev"] / pd.np.sqrt(tbl["N"]))
tbl["Sh"] = tbl["Mean"] / tbl["Stdev"]
round(tbl.drop("N", axis=1).transpose(), 2)

In [ ]:
(models / 100 + 1).cumprod().plot()

In [ ]:
(models[:"1999"] / 100 + 1).cumprod().plot()

In [ ]:
(models["2000":] / 100 + 1).cumprod().plot()

In [ ]:
(models["1963":"1969"] / 100 + 1).cumprod().plot()

In [ ]:
(models["1970":"1979"] / 100 + 1).cumprod().plot()

In [ ]:
(models["1980":"1989"] / 100 + 1).cumprod().plot()

In [ ]:
(models["1990":"1999"] / 100 + 1).cumprod().plot()

In [ ]:
(models["2000":"2009"] / 100 + 1).cumprod().plot()

In [ ]:
(models["2010":"2016"] / 100 + 1).cumprod().plot()

In [ ]:
R = F["1996":"2016"][bs2015].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
BS2015 = pd.Series(res["x"] @ R.transpose(), index=F["1996":"2016"].index)
bs2015, res["x"].round(2)

In [ ]:
R = F["1996":"2016"][ff2015].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
FF2015 = pd.Series(res["x"] @ R.transpose(), index=F["1996":"2016"].index)
ff2015, res["x"].round(2)

In [ ]:
R = F["1996":"2016"][ff2016a].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
FF2016a = pd.Series(res["x"] @ R.transpose(), index=F["1996":"2016"].index)
ff2016a, res["x"].round(2)

In [ ]:
R = F["1996":"2016"][ff2016b].as_matrix()
res = minimize(obj_SR, [.2,.2,.2,.2,.2], args=(R,), constraints=cons)
FF2016b = pd.Series(res["x"] @ R.transpose(), index=F["1996":"2016"].index)
ff2016b, res["x"].round(2)

In [ ]:
models = pd.DataFrame(
    {"BS2015": BS2015,
     "FF2015": FF2015,
     "FF2016a": FF2016a,
     "FF2016b": FF2016b,
     "Rm": F["1996":"2016"]["rm"]}
)

In [ ]:
(models / 100 + 1).cumprod().plot()

## Spanning Regressions

### FF2016 Operating Profit

$R^M + SMB + HML + RMW^r + CMA$

In [ ]:
factors = ff2016a
end = "2015"
contrib = []

for LHS in factors:
    RHS = [RHS for RHS in factors if RHS!=LHS]
    model = sm.OLS(F[:end][LHS], sm.add_constant(F[:end][RHS]))
    fit = model.fit()

    contrib.append(pd.Series({"name": LHS,
                    "a": fit.params["const"],
                    "Stdev": fit.resid.std(),
                    "contrib": fit.params["const"]/fit.resid.std()}))

pd.DataFrame(contrib).set_index("name")[["a", "Stdev", "contrib"]].transpose().round(3)

### FF2016b Cash Profit

$R^M + SMB + HML + RMW^c + CMA$

In [ ]:
factors = ff2016b  #  + ["wml",]
end = "2016"
contrib = []

for LHS in factors:
    RHS = [RHS for RHS in factors if RHS!=LHS]
    model = sm.OLS(F[:end][LHS], sm.add_constant(F[:end][RHS]))
    fit = model.fit()

    contrib.append(pd.Series({"name": LHS,
                    "a": fit.params["const"],
                    "SD": fit.resid.std(),
                    "N": len(fit.resid)}))

contrib = pd.DataFrame(contrib).set_index("name")  #[["a", "Stdev", "contrib", "contrib2"]].transpose().round(2)
contrib["SE"] = contrib["SD"] / contrib["N"]**(1/2)
contrib["a/SE"] = contrib["a"] / contrib["SE"]  # From table description
contrib["a/SD"] = contrib["a"] / contrib["SD"]  # From text description (sec 1)
contrib["(a/SE)^2"] = contrib["a/SE"]**2
contrib["(a/SD)^2"] = contrib["a/SD"]**2
contrib[["a", "SD", "(a/SE)^2", "(a/SD)^2"]].transpose().round(3)

### BS2015

$R^M + SMB + HML^m + RMW^c + WML$

In [ ]:
factors = bs2015  #  + ["cma",]
end = "2016"
contrib = []

for LHS in factors:
    RHS = [RHS for RHS in factors if RHS!=LHS]
    model = sm.OLS(F[:end][LHS], sm.add_constant(F[:end][RHS]))
    fit = model.fit()

    contrib.append(pd.Series({"name": LHS,
                    "a": fit.params["const"],
                    "SD": fit.resid.std(),
                    "N": len(fit.resid)}))

contrib = pd.DataFrame(contrib).set_index("name")  #[["a", "Stdev", "contrib", "contrib2"]].transpose().round(2)
contrib["SE"] = contrib["SD"] / contrib["N"]**(1/2)
contrib["a/SE"] = contrib["a"] / contrib["SE"]  # From table description
contrib["a/SD"] = contrib["a"] / contrib["SD"]  # From text description (sec 1)
contrib["(a/SE)^2"] = contrib["a/SE"]**2
contrib["(a/SD)^2"] = contrib["a/SD"]**2
contrib[["a", "SD", "(a/SE)^2", "(a/SD)^2"]].transpose().round(3)

In [ ]:
LHS = "cma"
RHS = bs2015

model = sm.OLS(F[:end][LHS], sm.add_constant(F[:end][RHS]))
fit = model.fit()
pd.Series({"name": LHS,
 "a": fit.params["const"],
 "Stdev": fit.resid.std(),
 "contrib": (fit.params["const"]/fit.resid.std())**2})

## Cumulative Returns

### Whole Sample

### 2000-

## Anomalies

In [4]:
def GRS(R, F, factors, start="1963", end="2016"):
    """
    Returns the GRS statistic of Gibbons, Ross and Shanken (1989)
    given test assets R and factors F.
    
    R is expected to be 
    """

    alphas = []
    errors = []

    for p in R.columns:
        model = sm.OLS(R[start:end][p] - F[start:end]["rf"],
                       sm.add_constant(F[start:end][factors]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        errors.append(fit.resid)

    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()

    K = F[factors].shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)

    r = F[factors].mean().as_matrix()

    f = F[factors].as_matrix()

    Vf = (f-r).transpose().dot(f-r) / (T - 1)

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f

    return (T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))

In [6]:
def model_coef_list(factors, LHS, end, k=5):

    coef_list = []
    coef_names = ["a",] + factors
    alphas = []

    for coef in coef_names:
        coef_list.append({"name": coef, "coef": [], "tval": []})

    for col in LHS.columns:

        # regress the "anomaly" on the factors
        model = sm.OLS(LHS["1963-07":end][col]-F["1963-07":end]["rf"],
                       sm.add_constant(F["1963-07":end][factors]))
        fit = model.fit()
        # get the coefficients and their p-values
        col_coef = list(fit.params)
        col_tval = list(fit.tvalues)

        for i, coef in enumerate(coef_names):
            coef_list[i]["coef"].append(round(col_coef[i], 2))
            coef_list[i]["tval"].append(round(col_tval[i], 2))
            
            if coef_list[i]["name"]=="a":
                alphas.append(col_coef[i])

    for i, coef in enumerate(coef_names):
        coef_list[i]["coef"] = pd.np.array(coef_list[i]["coef"]).reshape(k,k).tolist()
        coef_list[i]["tval"] = pd.np.array(coef_list[i]["tval"]).reshape(k,k).tolist()
    
    return coef_list, alphas

In [7]:
import jinja2
import os

path = "C:/Users/samth/Dropbox/Thesis/Tex/Tables/"

env = jinja2.Environment(
    block_start_string='-%', block_end_string='%-',
    variable_start_string='=%', variable_end_string='%=',
    loader=jinja2.FileSystemLoader(path)
)

In [8]:
template = env.get_template("5x5_template.txt")

### Size-Value (Replication)

#### French Dartmouth Anomaly

In [9]:
size_value = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV")
size_value = size_value.iloc[:642, 1:]
size_value.index = F.index

In [10]:
a = size_value
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'3.20, 2.27, 2.21, 2.66'

#### FF2015

In [11]:
coef_list, alphas = model_coef_list(ff2015, size_value, "2013")

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV")
# a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

factors = ff2015
LHS = a
start, end = "1963-07", "2013-12"
precision = 2

K = 2
j = 4

transpose_j = True

coef_names = ["a", "b", "s", "v", "p", "i"]


N = len(LHS.columns)
A = dict(
    zip(
        coef_names,
        [[[[[], []] for k in range(K)] for row in range(j)] for i in range(len(coef_names))]
    )
)

for lhsi, lhs in enumerate(LHS.columns):

    lhs = LHS[start:end][lhs] - F[start:end]["rf"]
    rhs = sm.add_constant(F[start:end][factors])

    # regress the "anomaly" on the factors
    model = sm.OLS(lhs, rhs)
    fit = model.fit()

    # get the coefficients and their t-values
    coef = [round(b, precision) for b in fit.params]
    tval = [round(t, precision) for t in fit.tvalues]
    
    for namei, name in enumerate(coef_names):

        k = int(lhsi//(N/K))
        A[name][k][0].append(coef[namei])
        A[name][k][1].append(tval[namei])

for namei, name in enumerate(coef_names):
    for k, coef in enumerate(A[name]):
        coef = pd.np.array(A[name][k][0]).reshape(j, j)
        if transpose_j: coef = coef.transpose()
        A[name][k][0] = coef.tolist()
        
        tval = pd.np.array(A[name][k][1]).reshape(j, j)
        if transpose_j: tval = tval.transpose()
        A[name][k][1] = tval.tolist()

In [96]:
# For a 2x4x4, I want 4 rows of two pairs of lists
[[[[], []] for k in range(K)] for i in range(len(coef_names))]

[[[[], []], [[], []]],
 [[[], []], [[], []]],
 [[[], []], [[], []]],
 [[[], []], [[], []]],
 [[[], []], [[], []]],
 [[[], []], [[], []]]]

In [101]:

dict(
    zip(
        coef_names,
        [[[[[], []] for k in range(K)] for row in range(j)] for i in range(len(coef_names))]
    )
)


{'a': [[[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]]],
 'b': [[[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]]],
 'i': [[[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]]],
 'p': [[[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]]],
 's': [[[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]]],
 'v': [[[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]],
  [[[], []], [[], []]]]}

In [93]:
A

{'a': [[[[0.03, -0.02, 0.16, 0.12],
    [0.13, -0.0, 0.03, 0.0],
    [-0.15, 0.15, 0.04, 0.13],
    [-0.49, -0.2, -0.08, -0.11]],
   [[0.45, -0.27, 1.95, 1.42],
    [1.91, -0.0, 0.5, 0.04],
    [-1.84, 2.43, 0.8, 2.44],
    [-6.06, -2.64, -1.48, -2.14]]],
  [[[0.05, -0.12, 0.0, -0.11],
    [-0.24, -0.08, 0.07, 0.01],
    [0.05, -0.06, -0.08, 0.03],
    [0.17, -0.26, 0.07, 0.33]],
   [[0.67, -1.56, 0.06, -1.35],
    [-2.49, -1.1, 0.99, 0.16],
    [0.62, -0.64, -1.02, 0.42],
    [2.01, -2.88, 0.81, 3.93]]]],
 'b': [[[[1.1, 1.05, 1.02, 1.11],
    [1.0, 0.97, 0.9, 1.02],
    [1.05, 0.91, 0.92, 0.99],
    [1.06, 1.07, 1.06, 1.12]],
   [[62.44, 57.24, 50.69, 51.66],
    [61.33, 71.01, 68.22, 65.42],
    [51.55, 59.93, 72.3, 74.71],
    [54.03, 56.9, 75.85, 86.2]]],
  [[[1.13, 1.01, 1.02, 1.04],
    [1.02, 0.9, 0.96, 0.94],
    [1.04, 1.06, 0.99, 0.96],
    [1.01, 1.1, 1.07, 1.03]],
   [[59.08, 53.57, 49.97, 51.45],
    [43.56, 49.6, 54.24, 49.95],
    [49.26, 50.4, 54.29, 53.33],
    [49.44,

In [55]:
dict(zip(coef_names, [[([], []) for n in range(1)] for i in range(len(coef_names))]))

{'a': [([], [])],
 'b': [([], [])],
 'i': [([], [])],
 'p': [([], [])],
 's': [([], [])],
 'v': [([], [])]}

1

In [12]:
coef_list

[{'coef': [[-0.29, 0.11, -0.03, 0.17, 0.12],
   [-0.11, -0.06, 0.0, 0.02, -0.04],
   [0.02, 0.01, -0.09, -0.01, 0.0],
   [0.18, -0.21, -0.18, 0.05, -0.11],
   [0.12, -0.1, -0.09, -0.26, -0.02]],
  'name': 'a',
  'tval': [[-3.17, 1.73, -0.45, 3.03, 2.03],
   [-1.67, -1.13, 0.08, 0.31, -0.61],
   [0.33, 0.22, -1.35, -0.11, 0.05],
   [2.76, -3.05, -2.52, 0.72, -1.19],
   [2.59, -1.74, -1.29, -3.82, -0.2]]},
 {'coef': [[1.04, 0.94, 0.94, 0.88, 0.98],
   [1.09, 1.02, 0.98, 0.98, 1.09],
   [1.06, 1.05, 1.01, 1.01, 1.11],
   [1.05, 1.11, 1.09, 1.04, 1.18],
   [0.97, 1.03, 0.98, 1.05, 1.08]],
  'name': 'rm',
  'tval': [[47.14, 57.85, 68.84, 63.78, 66.77],
   [70.3, 75.22, 73.21, 78.69, 77.45],
   [70.55, 66.16, 64.71, 66.02, 59.5],
   [66.15, 64.98, 61.97, 61.27, 53.98],
   [86.66, 73.36, 55.16, 63.54, 44.74]]},
 {'coef': [[1.24, 1.23, 1.08, 1.06, 1.09],
   [0.96, 0.93, 0.83, 0.74, 0.9],
   [0.7, 0.61, 0.5, 0.46, 0.61],
   [0.36, 0.28, 0.26, 0.23, 0.31],
   [-0.21, -0.14, -0.2, -0.16, -0.12]],

In [ ]:
context = {
    "var_name": "B/M",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Value Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2013 (606 Months).}",
    "label": "tbl:25_Size_Value_FF2015_FF"
}

In [ ]:
with open(os.path.join(path, "25_Size_Value_FF2015_FF_196307_201312.tex"), "w") as table:
    table.write(template.render(context))

#### My Anomaly

In [ ]:
size_value = pd.read_csv("C:/Data/Thesis/ME_BM_al_25_Returns.csv")
size_value = size_value.iloc[:642, 1:]
size_value.index = F.index

In [ ]:
# I am missing values for all of 1963 for the Small-LoBM Portfolio
size_value = size_value.fillna(0)

#### FF2015

In [ ]:
coef_list, alphas = model_coef_list(["rm", "SMB", "HMLo", "RMW", "CMA"], size_value, "2013")

In [ ]:
context = {
    "var_name": "B/M",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Replicated Size-Value Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2013 (606 Months).}",
    "label": "tbl:25_Size_Value_FF2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_Value_FF2015_196307_201312.tex"), "w") as table:
    table.write(template.render(context))

### Size-Investment

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

In [ ]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

#### FF2015

In [ ]:
coef_list, alphas = model_coef_list(ff2015, a, "2016")

In [ ]:
context = {
    "var_name": "Inv",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Inv Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Inv_FF2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_Inv_FF2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016a

In [ ]:
GRS(a, F, bs2015)

In [ ]:
coef_list, alphas = model_coef_list(ff2016a, a, "2016")

In [ ]:
context = {
    "var_name": "Inv",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Inv Portfolios with FF2016a Factors.\footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Inv_FF2016a"
}

In [ ]:
with open(os.path.join(path, "25_Size_Inv_FF2016a_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016b

In [ ]:
coef_list, alphas = model_coef_list(ff2016b, a, "2016")

In [ ]:
context = {
    "var_name": "Inv",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Inv Portfolios with FF2016b Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Inv_FF2016b"
}

In [ ]:
with open(os.path.join(path, "25_Size_Inv_FF2016b_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### BS2015

In [ ]:
coef_list, alphas = model_coef_list(bs2015, a, "2016")

In [ ]:
context = {
    "var_name": "Inv",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "tbl": coef_list,
    "caption": r"Size-Inv Portfolios with BS2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Inv_BS2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_Inv_BS2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

### Size-Volatility

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

In [ ]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

#### FF2015

In [ ]:
coef_list, alphas = model_coef_list(ff2015, a, "2016")

In [ ]:
context = {
    "var_name": "Vol",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Vol Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Vol_FF2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_Vol_FF2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016a

In [ ]:
coef_list, alphas = model_coef_list(ff2016a, a, "2016")

In [ ]:
context = {
    "var_name": "Vol",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Vol Portfolios with FF2016a Factors.\footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Vol_FF2016a"
}

In [ ]:
with open(os.path.join(path, "25_Size_Vol_FF2016a_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016b

In [ ]:
coef_list, alphas = model_coef_list(ff2016b, a, "2016")

In [ ]:
context = {
    "var_name": "Vol",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Vol Portfolios with FF2016b Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Vol_FF2016b"
}

In [ ]:
with open(os.path.join(path, "25_Size_Vol_FF2016b_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### BS2015

In [ ]:
coef_list, alphas = model_coef_list(bs2015, a, "2016")

In [ ]:
context = {
    "var_name": "Vol",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "tbl": coef_list,
    "caption": r"Size-Vol Portfolios with BS2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Vol_BS2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_Vol_BS2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

### Size-Volatility

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

In [ ]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

#### FF2015

In [ ]:
coef_list, alphas = model_coef_list(ff2015, a, "2016")

In [ ]:
context = {
    "var_name": "Mom",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Mom Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Mom_FF2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_Mom_FF2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016a

In [ ]:
coef_list, alphas = model_coef_list(ff2016a, a, "2016")

In [ ]:
context = {
    "var_name": "Mom",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Mom Portfolios with FF2016a Factors.\footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Mom_FF2016a"
}

In [ ]:
with open(os.path.join(path, "25_Size_Mom_FF2016a_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016b

In [ ]:
coef_list, alphas = model_coef_list(ff2016b, a, "2016")

In [ ]:
context = {
    "var_name": "Mom",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-Mom Portfolios with FF2016b Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Mom_FF2016b"
}

In [ ]:
with open(os.path.join(path, "25_Size_Mom_FF2016b_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### BS2015

In [ ]:
coef_list, alphas = model_coef_list(bs2015, a, "2016")

In [ ]:
context = {
    "var_name": "Mom",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "tbl": coef_list,
    "caption": r"Size-Mom Portfolios with BS2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_Mom_BS2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_Mom_BS2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

### Size-Profit

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_OP_5x5.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

In [ ]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

#### FF2015

In [ ]:
coef_list, alphas = model_coef_list(ff2015, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_FF2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_FF2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016a

In [ ]:
coef_list, alphas = model_coef_list(ff2016a, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with FF2016a Factors.\footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_FF2016a"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_FF2016a_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016b

In [ ]:
coef_list, alphas = model_coef_list(ff2016b, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with FF2016b Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_FF2016b"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_FF2016b_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### BS2015

In [ ]:
coef_list, alphas = model_coef_list(bs2015, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with BS2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_BS2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_BS2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

### Size-Profit-Inv

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

#A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

# 2x4x4 TEMPLATE

#### FF2015

In [ ]:
coef_list, alphas = model_coef_list(ff2015, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_FF2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_FF2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016a

In [ ]:
coef_list, alphas = model_coef_list(ff2016a, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with FF2016a Factors.\footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_FF2016a"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_FF2016a_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### FF2016b

In [ ]:
coef_list, alphas = model_coef_list(ff2016b, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with FF2016b Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_FF2016b"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_FF2016b_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

#### BS2015

In [ ]:
coef_list, alphas = model_coef_list(bs2015, a, "2016")

In [ ]:
context = {
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "tbl": coef_list,
    "caption": r"Size-OP Portfolios with BS2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:25_Size_OP_BS2015"
}

In [ ]:
with open(os.path.join(path, "25_Size_OP_BS2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

# All Anomalies

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

A = a

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_OP_2x4x4.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_OP_5x5.csv")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.csv")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_AC_5x5.csv")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.csv")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_RESVAR_5x5.csv")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.csv")
a = a.iloc[:642, 1:]
a.index = F.index

A = A.merge(a, left_index=True, right_index=True)

"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

In [ ]:
A.shape

In [ ]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(A, F, ff2015), GRS(A, F, ff2016a), GRS(A, F, ff2016b), GRS(A, F, bs2015))